In [ ]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict #순서까지 기억하는 사전형
import os, random
import pickle
# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz
# resnet
#http://laonple.blog.me/220761052425
#http://laonple.blog.me/220770760226

In [ ]:
# 파라미터들
input_size = 32
data_path = 'cifar10/cifar-10-batches-py/'

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']


list_class=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
n_side = 32
n_size = n_side * n_side
n_channel = 3

n_class = len(list_class)
n_layer = 3 #simple resnet

n_batch = 100

#input & output 
x = tf.placeholder("float", [n_batch, n_size * n_channel]) #batch x image size
y = tf.placeholder("float", [n_batch, n_class]) #batch x class

#weight&bias
stddev = 0.1
weights = {
    'c': tf.Variable(tf.random_normal([3, 3, n_channel, 16], stddev=stddev)),
    'c1_1': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'd1' :tf.Variable(tf.random_normal([16*16*64, 1000], stddev=stddev)),
    'd2' :tf.Variable(tf.random_normal([1000, n_class], stddev=stddev))
}
biases = {
    'c': tf.Variable(tf.random_normal([16], stddev=stddev)), 
    'c1_1': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([1000], stddev=stddev)),
    'd2': tf.Variable(tf.random_normal([n_class], stddev=stddev))
}

#시작은 맥스풀
#끝은 에버리지 풀 
# 3x3 conv 2개마다 2개 이전의 결과를 더해줌 
#마지막은 Fc1000

In [ ]:
#model
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='Simple_ResNet'):
    _names = OrderedDict()
    print('img_width')
    print(img_width)
    print('img_height')
    print(img_height)
    print('img_channel')
    print(img_channel)
    print(_x.shape)

    #reshape
    _x_r = tf.reshape(_x, shape=[-1, img_width, img_height, img_channel])
    
    with tf.variable_scope(scope):
        #max pool
        _pool = tf.nn.max_pool(_x_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        _names['pool'] = _pool
        
        #convolution
        _conv = tf.nn.conv2d(_pool, _w['c'], strides=[1,1,1,1], padding='SAME')
        _conv = tf.nn.bias_add(_conv, _b['c'])
        _conv = tf.nn.relu(_conv)
        _names['conv'] = _conv
        
        #residual conv - 1
        conv1_1 = tf.nn.conv2d(_conv, _w['c1_1'], strides=[1,1,1,1], padding='SAME')
        conv1_1 = tf.nn.bias_add(conv1_1, _b['c1_1'])
        conv1_1 = tf.nn.relu(conv1_1)
        _names['conv1_1'] = conv1_1
        
        conv1_2 = tf.nn.conv2d(conv1_1, _w['c1_2'], strides=[1,1,1,1], padding='SAME')
        conv1_2 = tf.nn.bias_add(conv1_2, _b['c1_2'])
        conv1_2 = tf.add(conv1_2 , _conv) #residual? 
        conv1_2 = tf.nn.relu(conv1_2)
        _names['conv1_2'] = conv1_2
        
        conv1_3 = tf.nn.conv2d(conv1_2, _w['c1_3'], strides=[1,1,1,1], padding='SAME')
        conv1_3 = tf.nn.bias_add(conv1_3, _b['c1_3'])
        conv1_3 = tf.nn.relu(conv1_3)
        _names['conv1_3'] = conv1_3
        
        conv1_4 = tf.nn.conv2d(conv1_3, _w['c1_4'], strides=[1,1,1,1], padding='SAME')
        conv1_4 = tf.nn.bias_add(conv1_4, _b['c1_4'])
        conv1_4 = tf.add(conv1_4 , conv1_2) #residual? 
        conv1_4 = tf.nn.relu(conv1_4)
        _names['conv1_4'] = conv1_4
        
        #residual conv - 2
        conv2_1 = tf.nn.conv2d(conv1_4, _w['c2_1'], strides=[1,1,1,1], padding='SAME')
        conv2_1 = tf.nn.bias_add(conv2_1, _b['c2_1'])
        conv2_1 = tf.nn.relu(conv2_1)
        _names['conv2_1'] = conv2_1
        
        conv2_2 = tf.nn.conv2d(conv2_1, _w['c2_2'], strides=[1,1,1,1], padding='SAME')
        conv2_2 = tf.nn.bias_add(conv2_2, _b['c2_2'])
        #conv2_2 = tf.add(conv2_2, conv1_4)
        conv2_2 = tf.add(conv2_2, conv2_1)
        conv2_2 = tf.nn.relu(conv2_2)
        _names['conv2_2'] = conv2_2
        
        conv2_3 = tf.nn.conv2d(conv2_2, _w['c2_3'], strides=[1,1,1,1], padding='SAME')
        conv2_3 = tf.nn.bias_add(conv2_3, _b['c2_3'])
        conv2_3 = tf.nn.relu(conv2_3)
        _names['conv2_3'] = conv2_3
        
        conv2_4 = tf.nn.conv2d(conv2_3, _w['c2_4'], strides=[1,1,1,1], padding='SAME')
        conv2_4 = tf.nn.bias_add(conv2_4, _b['c2_4'])
        conv2_4 = tf.add(conv2_4, conv2_2)
        conv2_4 = tf.nn.relu(conv2_4)
        _names['conv2_4'] = conv2_4
        
        #residual conv - 3
        conv3_1 = tf.nn.conv2d(conv2_4, _w['c3_1'], strides=[1,1,1,1], padding='SAME')
        conv3_1 = tf.nn.bias_add(conv3_1, _b['c3_1'])
        conv3_1 = tf.nn.relu(conv3_1)
        _names['conv3_1'] = conv3_1
        
        conv3_2 = tf.nn.conv2d(conv3_1, _w['c3_2'], strides=[1,1,1,1], padding='SAME')
        conv3_2 = tf.nn.bias_add(conv3_2, _b['c3_2'])
        #conv3_2 = tf.add(conv3_2, conv2_4)
        conv3_2 = tf.add(conv3_2, conv3_1)
        conv3_2 = tf.nn.relu(conv3_2)
        _names['conv3_2'] = conv3_2
        
        conv3_3 = tf.nn.conv2d(conv3_2, _w['c3_3'], strides=[1,1,1,1], padding='SAME')
        conv3_3 = tf.nn.bias_add(conv3_3, _b['c3_3'])
        conv3_2 = tf.nn.relu(conv3_3)
        _names['conv3_3'] = conv3_3
        
        conv3_4 = tf.nn.conv2d(conv3_3, _w['c3_4'], strides=[1,1,1,1], padding='SAME')
        conv3_4 = tf.nn.bias_add(conv3_4, _b['c3_4'])
        conv3_4 = tf.add(conv3_4, conv3_2)
        conv3_4 = tf.nn.relu(conv3_4)
        _names['conv3_4'] = conv3_4
        
        
        
        #vectorize
        _dense = tf.reshape(conv3_4, [-1, _w['d1'].get_shape().as_list()[0]])
        _names['dense'] = _dense
        
        d1 = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
        _names['d1'] = d1
        print(d1.shape)
        
        _logit = tf.add(tf.matmul(d1, _w['d2']), _b['d2'])
        _names['logit'] = _logit
        print(_logit)
                
    return _names        

In [ ]:
#prediction
out = ResNet(n_side, n_side, n_channel, x, weights, biases, 'Simple_ResNet')
print(out)


In [ ]:
#cost function & loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))

In [ ]:
#optimizer
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [ ]:
#initializer
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

In [ ]:
print(y)

In [ ]:
print(out['dense'])

In [ ]:
#save
savedir = "nets/cnn_cifar10_resnet/"
saver = tf.train.Saver(max_to_keep=3)
save_step = 4
if not os.path.exists(savedir):
 os.makedirs(savedir)
print ("SAVER READY")

In [ ]:
print(out)

In [ ]:
def load_npz(file):
 
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        return dict

In [ ]:
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data'])
    start_idx = list(range(0, num_images, batch_size))
    end_idx = list(range(batch_size, num_images + 1, batch_size))
    if allow_small_batch and end_idx[-1] < num_images :
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

In [ ]:
def one_hot_encode(x, numclass):
    return np.eye(numclass)[x]

In [ ]:
training_epochs = 1000
batch_size = 100
disp_each = 10
disp_batch =100
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

In [ ]:
load_testdata=load_npz(test_file[0])

In [ ]:
#training
for epoch in range(training_epochs):
    
    sample_list_new = random.sample(train_file, len(train_file))
    nNumBatch = 0
    AvgBatchCost = 0
    
    for index, file_name in enumerate(sample_list_new):
        file_name = data_path+file_name
        load_data = load_npz(file_name)
        batch_index = make_batch_index(load_data, batch_size, True)
        
        for start, end in batch_index:
            print(str(start)+", "+str(end))
            batch_xs = load_data[b'data'][start:end]
            batch_ys = one_hot_encode(load_data[b'labels'][start:end], n_class)
            #print(batch_ys)
            #print(batch_xs, shape)
            feeds = {x: batch_xs, y: batch_ys}
            sess.run(optm, feed_dict = feeds)
            nNumBatch+=1
            tmp_cost, _ = sess.run([cost, optm], feed_dict = feeds)
            AvgBatchCost += tmp_cost
        
        if nNumBatch % disp_batch == 0:
            train_acc = sess.run(accr, feed_dict = feeds)
            print('\t[%d nNumBatch] train_cost = %g, acc = %g'%(nNumBatch, AvgBatchCost/nNumBatch, train_acc))
        print(nNumBatch)
             
    if epoch % disp_each == 0:
        num_batch = 0
        test_acc = 0
        avg_acc = 0
        load_test_data = load_npz(test_file[0])
        testdatalabel = one_hot_encode(load_testdata[b'labels'], n_class)
        test_batch_index = make_batch_index(load_data, batch_size, True)
                
        for start, end in test_batch_index:
            num_batch += 1
            batch_xs = load_testdata[b'data'][start:end]
            batch_ys = one_hot_encode(load_testdata[b'labels'][start:end],n_class)
            test_acc = sess.run(accr, feed_dict={x:batch_xs,y:batch_ys})
            avg_acc+=test_acc
        print('[%d epoch]test acc=%g'%(epoch, avg_acc/num_batch))

In [ ]:
print(test_batch_index)

In [ ]:
for start, end in test_batch_index:
    print(start, end)